# Predictions

## Imports

In [ ]:
import pandas as pd
import numpy as np

from tensorflow.data import Dataset

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from matplotlib import pyplot as plt

from keras import Input
from keras import Model
from keras import layers
from keras import models
from keras import losses
from keras.utils import to_categorical
from keras.preprocessing import timeseries_dataset_from_array

## Test whether Notebook is running on Google Colab

In [ ]:
csvPath = ''
if 'google.colab' in str(get_ipython()):
  csvPath = 'https://github.com/DridriLaBastos/Masterials/raw/main/PatientsHTA.zip'
else:
  csvPath = 'PatientsHTA.zip'

In [ ]:
dateColumnNames = [
    'contact_date',
    'Glycemie_der_date',
    'HbA1c_der_date',
    'der_date_poids',
    'der_date_taille',
    'first_contact_date'
]

df = pd.read_csv(csvPath,engine='c',parse_dates=dateColumnNames)

# Suppression des lignes trop peu nombreuses

Nous souhaitons faire un apprentissage en utilisant la dimension temporelle comme filtre pour le CNN. Pour ça il faut donc que nous ayons plusieurs entrées. Avant de commencer à traîter les données, nous supprimons toutes les personnes qui n'ont pas rendu visite assez souvent à leur médecin. Ainsi, par le biais de ```person_id```, nous avons choisi arbitrairement que pour être utile à l'apprentissage, il faut au moins 3 visites par patients, soient toutes les lignes dont le ```person_id```est contenu plus de 3 fois dans tout le jeu de données.

In [ ]:
VISIT_NUMBER = 4
valueCounts = df.person_id.value_counts()
dfEnought = df[df.person_id.isin(valueCounts[valueCounts.values >= VISIT_NUMBER].index)]

# Suppression des colonnes innutiles

## Suppression de la colonne ```age_now```

Nous pouvons supprimer la colonne ```age_now``` car les données qu'elle contient sont identiques à celles de la colonne ```year_of_birth```.

In [ ]:
dfWithoutAgeNow = dfEnought.drop('Age_now', axis='columns')

## Suppression de la colonne ```contact_id```

En effet, la colonne ```contact_id``` ne représente aucun intérêt pour l'apprentissage car elle ne contient aucun information à même d'influer sur la prédiction.

In [ ]:
dfWithoutContactID = dfWithoutAgeNow.drop('contact_id',axis='columns')

## Suppression des noms de médicaments

In [ ]:
dfGroupedByMoleculeLabel = dfWithoutContactID.groupby('product_atc_code')[['molecule_label','short_name','long_name','Classe','product_atc']].count()
dfGroupedByMoleculeLabel

Nous voyons qu'il existe différentes colonnes dont le but est de désigner le médicament prescrit lors de la visite, or nous n'avons besoin que d'une seule colonne garder cette information. De ce fait, nous avons choisi de garder ```product_atc_code```.

In [ ]:
dropColumnNames = dfGroupedByMoleculeLabel.columns.to_list()
dfWithATCCode = dfWithoutContactID.drop(dropColumnNames, axis='columns')
dfWithATCCode

## Suppression des colonnes ```'*der*'```

Les colonnes ```'*der*'``` contiennent la dernière donnée. Cette donnée peut être récupérée grâce à la date de la visite et aux valeurs mesurées. Par exemple, il n'est pas nécessaire d'avoir une colonne ```der_date``` ou ```der_mesure```. Les données de ces deux types de colonnes peuvent être récupérées grâce à la ligne qui correspond à la dernière date de la mesure, que nous pouvons trouver grâce à la colonne ```contact_date```.

In [ ]:
derColumnNames = []

for c in dfWithATCCode.columns:
    if ('der_date' in c) or ('der_mesure' in c):
        derColumnNames.append(c)

dfWithoutDer = dfWithATCCode.drop(derColumnNames,axis='columns')

## Suppression des colonnes ```Taille``` et ```Poids```

In [ ]:
print(f"Taille: {dfWithoutDer.Taille.isnull().sum()}/{len(dfWithoutDer.Taille)} valeurs nulles (={dfWithoutDer.Taille.isnull().sum()/len(dfWithoutDer.Taille)*100:.2f}%)")

print(f"Poids: {dfWithoutDer.Poids.isnull().sum()}/{len(dfWithoutDer.Poids)} valeurs nulles (={dfWithoutDer.Poids.isnull().sum()/len(dfWithoutDer.Poids)*100:.2f}%)".format())

Nous voyons qu'il y a beaucoup trop de valeur nulles. Ces deux colonnes semblent donc difficilement exploitable. Nous pouvons cependant vérifier si pour les patients toutes les valeurs sont à nulles ou s'il n'existe que quelques entrées à nulle par patient mais qu'il y en a beaucoup. Dans ce cas nous pourrions enlever les lignes contenant des valeurs nulles, ou trouver un moyen d'attribuer une valeur à la place de Nan

In [ ]:
dfPersonIdIndex = dfWithoutDer.set_index('person_id',drop=True).sort_index()
dfTPGroupBy = dfPersonIdIndex.groupby('person_id')

In [ ]:
maybeUseful = 0
valeurNulle = 0
for i,_ in dfTPGroupBy:
    if dfPersonIdIndex.loc[i].Taille.isnull().sum() > 0:
        valeurNulle += 1
        if dfPersonIdIndex.loc[i].Taille.isnull().sum() < len(dfPersonIdIndex.loc[i].Taille):
            maybeUseful += 1

print(f"Taille: {maybeUseful} / {valeurNulle} utilisables")

maybeUseful = 0
valeurNulle = 0
c = 0
for i,_ in dfTPGroupBy:
    if dfPersonIdIndex.loc[i].Poids.isnull().sum() > 0:
        valeurNulle += 1
        if dfPersonIdIndex.loc[i].Poids.isnull().sum() < len(dfPersonIdIndex.loc[i].Poids):
            maybeUseful += 1

print(f"Poids: {maybeUseful} / {valeurNulle} utilisables")

In [ ]:
tailleNan = 0
poidsNan = 0
oneOfBoth = 0
bothNan = 0
totalEntries = 0
for i,_ in dfTPGroupBy:
    totalEntries += 1
    hasTailleNan = False
    hasPoidsNan = False
    if dfPersonIdIndex.loc[i].Taille.isnull().sum() != 0:
        tailleNan += 1
        hasTailleNan = True
    if dfPersonIdIndex.loc[i].Poids.isnull().sum() != 0:
        poidsNan += 1
        hasPoidsNan = True
    if hasTailleNan or hasPoidsNan:
        oneOfBoth += 1
    if hasTailleNan and hasPoidsNan:
        bothNan += 1
print(" --- Statistique par Utilisateur --- ")
print(f"{tailleNan} / {totalEntries} ({tailleNan/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour la taille")
print(f"{poidsNan} / {totalEntries} ({poidsNan/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour le poids")
print(f"{oneOfBoth} / {totalEntries} ({oneOfBoth/totalEntries*100:.2f}%) des utilisateurs ont une valeur nulle pour la taille ou le poids")
print(f"{bothNan} / {totalEntries} ({bothNan/totalEntries*100:.2f}%) des utilisateurs ont les deux valeurs nulle pour la taille ou le poids")



Nous concluons de l'analyse de ces données que soit toutes les valeurs de poids et de tailles sont entrées, soit aucunes. Cela rend ces informations innexploitables et nous supprimons donc les colonnes

In [ ]:
dfWithoutPT = dfWithoutDer.drop(['Taille', 'Poids'],axis='columns')

## Suppressions diverses

Enfin, certaines colonnes n'apportent pas d'informations nécessaires pour la prédiction, nous choisissons de toutes les supprimer ici

In [ ]:
dfWithoutPT.isnull().sum()

Les colonnes restantes avec des valeurs ```Nan``` ne nous intéresse pas, nous pouvons les supprimer

In [ ]:
nullAmount = dfWithoutPT.isnull().sum()

columnNameToDrop = nullAmount[nullAmount.values > 0].index
dfFinal = dfWithoutPT.drop(columnNameToDrop,axis='columns').drop(['cip','dosage_1','dose_1','dose_2','specialty_label','gender_code'],axis='columns')

# Traîtement des données

## Conversion des données

### Ajout du temps entre chaque visite (ce que l'on veut prédire)

Nous créons d'abord la colonne ```wait_time``` pour qu'elle ait le type de donnée ```deltatime```. nous itèrerons plus tard sur chaque valeur de cette colonne pour lui enlever la valeur précédante pour chaque utilisateur, et ainsi avoir l'intervalle de temps entre chaque visite

In [ ]:
wait_time = dfFinal.contact_date - dfFinal.first_contact_date
dfWithTime = dfFinal.drop('first_contact_date',axis='columns')
dfWithTime['wait_time'] = wait_time
dfWithTime['contactDateYear'] = dfWithTime.contact_date.dt.year
dfWithTime['contactDateMonth'] = dfWithTime.contact_date.dt.month
dfWithTime['contactDateDayOfYear'] = dfWithTime.contact_date.dt.dayofyear

In [ ]:
dfWithTime

### Encodage des valeurs non numériques

In [ ]:
#specialtyEncoder = LabelEncoder()
ATCEncoder = LabelEncoder()
#genderEncoder = LabelEncoder()
#dfWithTime.specialty_label = specialtyEncoder.fit_transform(dfWithTime.specialty_label)
dfWithTime.product_atc_code = ATCEncoder.fit_transform(dfWithTime.product_atc_code)
#dfWithTime.gender_code = genderEncoder.fit_transform(dfWithTime.gender_code)


In [ ]:
dfWithTime.hist()

### Conversion en ```TimeSeries```

Nous définissons simplement le nouvel index comme la colonne donnant l'intervalle de temps entre chaque visite.

In [ ]:
ts = dfWithTime.set_index(['person_id','contact_date']).sort_index()
ts

### Attribution des bonnes valeurs de ```time_wait```

In [ ]:
tsWithTime = ts.copy()
for i,_ in ts.groupby('person_id'):
    len_ = len(ts.loc[i])
    tsWithTime.loc[i].wait_time[1:len_] = pd.Series(ts.loc[i].wait_time.to_numpy()[1:len_] - ts.loc[i].wait_time.to_numpy()[:len_-1])
    tsWithTime.loc[i].wait_time[0] = pd.Timedelta(0)

In [ ]:
tsWithTimeNumber = tsWithTime.copy()
tsWithTimeNumber.wait_time = tsWithTime.wait_time.dt.days
tsWithTimeNumber['wait_time_days'] = tsWithTime.wait_time.dt.days
tsWithTimeNumber['wait_time_weeks'] = (tsWithTime.wait_time.dt.days / 7).astype(int)
tsWithTimeNumber.wait_time.describe()

In [ ]:
tsWithGoodTime = tsWithTimeNumber[tsWithTimeNumber.wait_time.values <= 300]
tsWithGoodTime.wait_time.describe()

### Transformation de ```wait_time``` en valeur numérique

In [ ]:
tsWithTimeMonth = tsWithGoodTime.copy()
tsWithTimeMonth.wait_time = (tsWithGoodTime.wait_time / 30).astype(int)
tsWithTimeMonth.wait_time[tsWithTimeMonth.wait_time >= 4] = 4
tsWithTimeMonth

In [ ]:
tsResetIndex = tsWithTimeMonth.copy()
tsResetIndex = tsResetIndex.reset_index()
print( tsResetIndex )
tsValueCounts = tsResetIndex.person_id.value_counts()
tsIndexed = tsResetIndex[tsResetIndex.person_id.isin(tsValueCounts[tsValueCounts.values >= VISIT_NUMBER].index)]

In [ ]:
atcCodeValues = tsWithTimeMonth.product_atc_code
waitTimeValues = tsWithTimeMonth.wait_time

tsFinal = pd.concat([tsWithTimeMonth,pd.get_dummies(atcCodeValues),pd.get_dummies(waitTimeValues)],axis='columns').drop(['product_atc_code','wait_time'],axis='columns')

# Prédiction

## Création des données d'entraînement/test

In [ ]:
#yColumnNames = ['product_atc_code', 'wait_time']

#xList,yList = [],[]
#for i,_ in tsWithTimeMonth.groupby('person_id'):
    # Mme ZERHAOUI a dit qu'il fallait transposer, je transpose
    # Pour l'instant je retourne au model évident : une série temporel qui contient 4 éléments de n_features données, on verra après pour la trasposition
#    currentSeriesX = tsFinal.loc[i]
#    currentSeriesY = tsWithTimeMonth.loc[i]
#    for j in range(0,len(currentSeriesX)-VISIT_NUMBER+1):
#        xList.append(currentSeriesX[j:j+VISIT_NUMBER-1].to_numpy().astype('float32'))
        #xList.append(currentSeries.to_numpy().astype('float32'))
#        yList.append(currentSeriesY[yColumnNames].values[j+VISIT_NUMBER-1].astype('float32'))
        #yList.append(currentSeries[yColumnNames].astype('float32'))

#xData = np.array(xList).reshape((len(xList),xList[0].shape[0],xList[0].shape[1]))
#yData = np.array(yList).reshape((len(yList),len(yColumnNames)))

In [ ]:
len(tsWithTimeMonth.groupby('person_id'))

In [ ]:
tsFinal.loc[291.0][:4]

In [ ]:
VISIT_THRESHOLD = VISIT_NUMBER

In [ ]:
yColumnNames = ['product_atc_code', 'wait_time']
xList,yList = [],[]
for i,_ in tsWithTimeMonth.groupby('person_id'):
    if(len(tsFinal.loc[i]) >= VISIT_THRESHOLD):
        xList.append(tsFinal.loc[i][:VISIT_THRESHOLD][:-1].to_numpy().astype('float32'))
        yList.append(tsWithTimeMonth[yColumnNames].loc[i][:VISIT_THRESHOLD][-1:].to_numpy().astype('float32'))

In [ ]:
xData = np.array(xList)
yData = np.array(yList)

xs = xData.shape
ys = yData.shape

xData = xData.reshape(xs[0],1,xs[1],xs[2])
yData = yData.reshape(ys[0],ys[1],ys[2])

In [ ]:
print(f"{xData.shape} --- {yData.shape}")

In [ ]:
l = len(xData) // 2
xData1, yData1 = xData[:l], yData[:l]
xData2, yData2 = xData[:-l], yData[:-l]

In [ ]:
trainUse1 = int(len(xData1) * 2 / 3)
testUse1 = len(xData1) - trainUse1

xTrain1, xTest1, yTrain1, yTest1 = xData1[:trainUse1],xData1[-testUse1:],yData1[:trainUse1],yData1[-testUse1:]

print(f"Train: {xTrain1.shape} --- Test: {xTest1.shape}")

In [ ]:
#Cellule pour faire des tests et comprendre la syntaxe que j'ai utilisée après
L = [[1,2,3,4,5,6],[7,8,9],[10,11]]
for l in L:
    # -2: permet d'avoir les deux derniers
    # :le-2 permet d'avoir toutes les entrées sauf les deux dernières
    print(f"{l[-2:]} | {l[:-2]}")

In [ ]:
print(xData2.shape, "   ", yData2.shape)

In [ ]:
xTrain2, yTrain2 = xData2, yData2

In [ ]:
tsFinal.loc[452710.0][VISIT_THRESHOLD:VISIT_THRESHOLD*2][:-1]

In [ ]:
xList2, yList2 = [],[]
for i,_ in tsWithTimeMonth.groupby('person_id'):
    if(len(tsFinal.loc[i]) >= VISIT_THRESHOLD*2):
        xList2.append(tsFinal.loc[i][VISIT_THRESHOLD:VISIT_THRESHOLD*2][:-1].to_numpy().astype('float32'))
        yList2.append(tsWithTimeMonth[yColumnNames].loc[i][VISIT_THRESHOLD:VISIT_THRESHOLD*2][-1:].to_numpy().astype('float32'))

In [ ]:
xTest2 = np.array(xList2)
yTest2 = np.array(yList2)

xs2 = xTest2.shape
ys2 = yTest2.shape

xTest2 = xTest2.reshape(xs2[0],1,xs2[1],xs2[2])
yTest2 = yTest2.reshape(ys2[0],ys2[1],ys2[2])

## Création du modèle

In [ ]:
ATC_CODE = 0
WAIT_TIME = 1
toTrain = WAIT_TIME

In [ ]:
#model = models.Sequential()
#model.add(layers.Conv1D(filters=256,kernel_size=VISIT_NUMBER-1, activation=None, input_shape=xData.shape[-3:]))
#model.add(layers.Attention())
#model.add(layers.Dense(units=32,activation='relu'))
#model.add(layers.LeakyReLU())
#model.add(layers.GaussianDropout(0.05))
#model.add(layers.Conv1D(filters=128,kernel_size=VISIT_NUMBER-1, padding="same", activation=None))
#model.add(layers.LeakyReLU())
#model.add(layers.GaussianDropout(0.05))
#model.add(layers.Conv1D(filters=128,kernel_size=VISIT_NUMBER-1, padding="same", activation=None))
#model.add(layers.LeakyReLU())
#model.add(layers.GaussianDropout(0.05))
#model.add(layers.Conv1D(filters=128,kernel_size=VISIT_NUMBER-1, padding="same", activation=None))
#model.add(layers.LeakyReLU())
#model.add(layers.GaussianDropout(0.05))
#model.add(layers.GlobalMaxPooling2D())
#if toTrain == ATC_CODE:
#    model.add(layers.Dense(units=len(ATCEncoder.classes_),activation='softmax'))
#elif toTrain == WAIT_TIME:
#    model.add(layers.Dense(units=tsWithTimeMonth.wait_time.max()+1,activation='softmax'))

In [ ]:
inputs = Input(shape=xData.shape[-3:])
x = layers.Conv1D(filters=256,kernel_size=VISIT_NUMBER-1, activation=None)(inputs)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(filters=256,kernel_size=VISIT_NUMBER-1, activation=None)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv1D(filters=256,kernel_size=VISIT_NUMBER-1, activation=None)(x)
x = layers.LeakyReLU()(x)
x = layers.GlobalMaxPooling2D()(x)
atcOutputs = layers.Dense(units=len(ATCEncoder.classes_),activation='softmax',name='outputATC')(x)
waitTimeOutputs = layers.Dense(units=tsWithTimeMonth.wait_time.max()+1,activation='softmax',name='outputWaitTime')(x)
model = Model(inputs=inputs,outputs=[atcOutputs,waitTimeOutputs],name="NiceCNN")

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

In [ ]:
yTrain = {
    'outputATC' : yTrain1[:,:,ATC_CODE],
    'outputWaitTime' : yTrain1[:,:,WAIT_TIME]
}

yTest = {
    'outputATC' : yTest1[:,:,ATC_CODE],
    'outputWaitTime' : yTest1[:,:,WAIT_TIME]   
}

In [ ]:
history = model.fit(xTrain1,yTrain,epochs=300,validation_data=(xTest1,yTest))
#history = model.fit(xTrain2,yTrain2,epochs=100,validation_data=(xTest2,yTest2))

In [ ]:
loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

## As loss always exists
epochs = range(1,len(history.history[loss_list[0]]) + 1)

fig, (p1,p2) = plt.subplots(1,2,figsize=(13,4))

## Loss
for l in loss_list:
    p1.plot(epochs, history.history[l], 'b', label='Training loss')
for l in val_loss_list:
    p1.plot(epochs, history.history[l], 'g', label='Validation loss')

p1.set(xlabel='Epochs',ylabel='Loss')

## Accuracy
for l in acc_list:
    p2.plot(epochs, history.history[l], 'b', label='Training accuracy')
for l in val_acc_list:    
    p2.plot(epochs, history.history[l], 'g', label='Validation accuracy')

p1.set(xlabel='Epochs',ylabel='Accuracy')
plt.legend()
plt.show()

In [ ]:
yTest = yTest1
predictions = np.argmax(model.predict(xTest1),axis=1).reshape(yTest[:,:,toTrain].shape)
print(f"Précision '{yColumnNames[toTrain]}': {(predictions == yTest[:,:,toTrain]).sum()/len(yTest)*100:.2f}%")

In [ ]:
SLICE_START = 200
SLICE_SIZE = 50
SLICE_END = SLICE_START + SLICE_SIZE

plt.plot(yTest[SLICE_START:SLICE_END,:,toTrain],label='expected')
plt.plot(predictions[SLICE_START:SLICE_END,0],color='r',label='predicted')

fig.legend()
fig.show()